In [2]:
import pandas as pd
import numpy as np
import snowflake.connector
import time

In [2]:
matched_list_with_temp_table = pd.read_csv('~/Downloads/matched_list.csv' , dtype = 'str')

SELECT GS1_COMPANY_PREFIX,GS1_COMPANY_NAME, COUNT(DISTINCT PRODUCT_MASTER_CATALOG_KEY)
FROM FDC_STAGE.DASCI.PRODUCT_MASTER_CATALOG p
JOIN FDC_STAGE.DASCI.PRODUCT_MASTER_CATALOG_MAPPED_TEMP pt on pt.GTIN = p.GTIN
GROUP BY GS1_COMPANY_PREFIX,GS1_COMPANY_NAME
ORDER BY COUNT(DISTINCT PRODUCT_MASTER_CATALOG_KEY) DESC

In [3]:
matched_list_with_temp_table

,GS1_COMPANY_PREFIX,GS1_COMPANY_NAME,COUNT(DISTINCT PRODUCT_MASTER_CATALOG_KEY)
0,0080686,JIM BEAM BRANDS CO,1345
1,0018200,ANHEUSER-BUSCH INBEV,1196
2,0085000,E & J GALLO WINERY INC.,916
3,0087692,BOSTON BEER CORPORATION,724
4,0096749,HEAVEN HILL DISTILLERIES INC.,565
...,...,...,...
4081,08663560001,THE SPIRIT GUILD LLC,1
4082,0896307002,CAMPBELL IMPORTS INC.,1
4083,0721512,FREY VINEYARDS LTD.,1
4084,08620690002,CAMPBELL LANE VENTURES LLC,1


In [4]:
company_prefix_name = pd.read_csv('~/Downloads/company_prefix_name.csv', dtype = 'str')

In [14]:
len(company_prefix_name)

9050

In [11]:
unmatched = company_prefix_name[~company_prefix_name['GS1_COMPANY_PREFIX'].isin(matched_list_with_temp_table['GS1_COMPANY_PREFIX'])]

In [16]:
len(unmatched)

4752

In [6]:
listofUPC_gt10 = pd.read_csv('~/Downloads/listofUPC_gt10.csv', dtype = 'str')
listofUPC_gt10

,INVOICE_UPC_CASE
0,00819902020104
1,00034100007812
2,00071990300173
3,00080660956190
4,00018200286117
...,...
370259,848169005913
370260,083089531358
370261,1085672400602
370262,81675022006


In [17]:
unmatched.isna().sum()

GS1_COMPANY_PREFIX    0
GS1_COMPANY_NAME      0
dtype: int64

In [18]:
unmatched.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Comparing unmatched list with listofUPC.

In [19]:
unmatched_list_of_temp_tables = []

for i in unmatched['GS1_COMPANY_PREFIX']:
    for j in listofUPC_gt10['INVOICE_UPC_CASE']:
        if i.strip("0")[:7] == j.strip("0")[:7] and i not in unmatched_list_of_temp_tables:
            unmatched_list_of_temp_tables.append(i)


In [30]:
unmatched_list_of_temp_tables_df = pd.DataFrame(unmatched_list_of_temp_tables,columns = ['GS1_COMPANY_PREFIX'])
unmatched_list_of_temp_tables_df

,GS1_COMPANY_PREFIX
0,08600036349
1,08600007186
2,0852882007
3,0850012046
4,08600033403
...,...
2588,803266513
2589,803277162
2590,8437008073
2591,08600010887


In [31]:
unmatced_to_matched = unmatched_list_of_temp_tables_df[unmatched_list_of_temp_tables_df['GS1_COMPANY_PREFIX'].isin(matched_list_with_temp_table['GS1_COMPANY_PREFIX'])]

In [67]:
unmatched_list_of_temp_tables_df.dropna(inplace = True)
matched_list_with_temp_table.dropna(inplace = True)

In [71]:
unmatced_to_matched_list = []

for i in unmatched_list_of_temp_tables_df['GS1_COMPANY_PREFIX']:
    iiii = i[:8]
    for j in matched_list_with_temp_table['GS1_COMPANY_PREFIX']:
        jjjj = j[:8]
        if iiii == jjjj and i not in unmatced_to_matched_list:
            unmatced_to_matched_list.append(i)


In [72]:
len(unmatced_to_matched_list)

2211

In [ ]:

cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_XS', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')

con = cnx.cursor()

select_query = f'''select * from  FDC_STAGE.DASCI.PRODUCT_MASTER_CATALOG_MAPPED_TEMP '''
results = con.execute(select_query)
PRODUCT_MASTER_CATALOG_MAPPED_TEMP = pd.DataFrame.from_records(iter(results), columns=[y[0] for y in results.description])


    

con.close()

